<a href="https://colab.research.google.com/github/Eugene31415/Pet-Projects/blob/main/LSB_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image # Library for image processing
import random # Random generator

In [ ]:
def string_to_bin_chain(message_string):
    chain = list() # Variable to store bin chain
    for character in message_string: # main loop. We iterate character by character through the text
        binary_string = '{0:08b}'.format(ord(character)) # Transfrom every letter to bin code (8 bit)
        for value in binary_string: # Append every of 8 bits into chain variable
            chain.append(int(value))
    return chain # Return bin sequence

In [ ]:
def bin_chain_to_string(chain, secure_anchor):
    message_string = '' # Result variable to store answer
    chain_pointer = 0
    while chain_pointer + 8 <= len(chain): # Iterate through all bin sequence with 8 bit step to read exactly one letter
        binary_string = ''.join(str(x) for x in chain[chain_pointer:chain_pointer + 8]) # Take 8 bits and concate into a string
        message_string += chr(int(binary_string, 2)) # Transform these 8 bits into a character
        chain_pointer += 8 # Increment by 8 such as there are exactly 8 bits to store one letter
        if len(message_string) > len(secure_anchor) and message_string[-(len(secure_anchor)):] == secure_anchor:
            break # Условие на то, сколько продолжать считывать и когда останавливаться
    return message_string[:-(len(secure_anchor))] # Возвращаем все сообщение, не включая защитный якорь

In [ ]:
def decode_pixel(rgba_tuple):
    binary_red = int('{0:08b}'.format(rgba_tuple[0])[-1], base=2) # Берем последний бит
    binary_green = int('{0:08b}'.format(rgba_tuple[1])[-1], base=2)
    binary_blue = int('{0:08b}'.format(rgba_tuple[2])[-1], base=2)
    binary_alpha = int('{0:08b}'.format(rgba_tuple[3])[-1], base=2)
    return [binary_red, binary_green, binary_blue, binary_alpha] # Возвращаем последние биты каждого канала

In [ ]:
# Function for reading text
def load_text(text_path, secure_anchor):
    with open(text_path, 'r', encoding='utf-8') as f: # Opens a file for reading with unf-8 encoding
        text = f.read() # Reads file and puts it into text variable
    text += secure_anchor # Places secure_anchor at the end of a string (for example ++++++++++)
    return text # Returns this modified text

In [ ]:
def decode_image(image_path, secure_anchor='++++++++++'):
    im = Image.open(image_path) # Open image
    im = im.convert("RGBA") # Convert from RGB to RGBA
    pixel_tuples = im.getdata() # Get 1d array of all pixels
    message_chain = list() # container for bits of decoded message
    for pixel_item in pixel_tuples: # Iterate through all pixels in the image
        message_chain_list = decode_pixel(pixel_item) # Получаем последние биты каждого канала в пикселе
        message_chain.append(message_chain_list[0]) # Добавляем их всех в ответ
        message_chain.append(message_chain_list[1])
        message_chain.append(message_chain_list[2])
        message_chain.append(message_chain_list[3])
    return bin_chain_to_string(message_chain, secure_anchor) # переводим в текст с помощью bin_chain_to_string и возвращаем

In [ ]:
'''У нас каждый пиксель представлен комбинацией из 4х цветов -RGBA, где А - это Альфа. Эта он всегда 255, то есть
самый яркий. В нашем случае эта штука расширяет кол-во цветовых каналов в каждом
пикселе. Теперь их не 3 как в RGB, а 4 - RGBA, что позволяет в каждый пиксель запульнуть 4 бита'''
def encode_pixel(rgba_tuple, message_slice):
    '''Первый аргумент - пиксели картинка, второй - 4 бита (в нашем случае) от бинарной цепочки'''
    binary_red = '{0:08b}'.format(rgba_tuple[0])[:7] + str(message_slice[0]) # берем первые 7 пикселей и добавляем 8 ой пиксель
    binary_green = '{0:08b}'.format(rgba_tuple[1])[:7] + str(message_slice[1])# Где 8ой пиксель это уже пискель текста 
    binary_blue = '{0:08b}'.format(rgba_tuple[2])[:7] + str(message_slice[2])
    binary_alpha = '{0:08b}'.format(rgba_tuple[3])[:7] + str(message_slice[3])
    return (
        int(binary_red, base=2), # переводим обратно в числа в 10ой системе и возвращаем кортеж
        int(binary_green, base=2),
        int(binary_blue, base=2),
        int(binary_alpha, base=2)
    )

In [ ]:
def encode_image(image_path, text_path, save_path, secure_anchor='++++++++++'):
    im = Image.open(image_path) # Read the image
    im = im.convert("RGBA") # Convert from RGB to RGBA where A means alpha
    pixel_tuples = im.getdata() # Creates a 1d vector of all pixels in the image  
    message = load_text(text_path, secure_anchor) # Reads the text message u want to put into an image
    message_chain = string_to_bin_chain(message) # Transforms message into bin chain (stores in list)
    message_pointer = 0 # pointer that controls borders of text message
    new_pixel_tuples = list() # Новые пиксели картинки, с учетом бинарной информации из текста
    for pixel_item in pixel_tuples:
        if message_pointer != len(message_chain): # If there are still no used bits from text message
            # Добавляем биты сообщения в пиксель и засовываем это в ответ
            new_pixel_tuples.append(encode_pixel(pixel_item, message_chain[message_pointer:message_pointer + 4]))

            message_pointer += 4 # этот указатель показывает на биты в сообщении и его двигаем на 4 позиции
        else: # если прочитали сообщение до конца, то в качестве последнего бита кладем рандомный
            noise_chain = [0, 0, 0, 0]
            for i in range(4):
                noise_chain[i] = random.randint(0, 1) # генерим рандомные биты 
            new_pixel_tuples.append(encode_pixel(pixel_item, noise_chain)) # Добавляем рандомные биты 
    im.putdata(new_pixel_tuples) # сохраняем все это говно с новыми битами как новую картинку, поверх старой
    im.save(save_path, "PNG") # Сохраняем это по адресу save_path с расширением PNG 

In [ ]:
# Program start
print('Start program')
print(10*'-')
encode_image('C:/Users/Xiaomi/Pictures/cat.png', 'C:/Users/Xiaomi/Pictures/text.txt', 'C:/Users/Xiaomi/Pictures/cat1.png')
print('Finish encode')
print(10*'-')
print('Start decode:')
print(10*'-')
print(decode_image('C:/Users/Xiaomi/Pictures/cat1.png'))

Start program
----------
Finish encode
----------
Start decode:
----------
sfghsdjgbsjkfgbzsjkfhbvzjsbvsjkrvbjkbvggjkls
